# 흑백 이미지 컬러로 복원

#### 패키지 로드하기

In [ ]:
# 패키지 로드하기
import cv2
import numpy as np

: 

In [ ]:
img = cv2.imread('./semi-project/img/1900s NewYork.jpg', cv2.IMREAD_UNCHANGED)
dst = cv2.resize(img, dsize=(800, 600), interpolation = cv2.INTER_AREA)

#cv2.imshow('img', img)
cv2.imshow("dst", dst)

cv2.waitKey()
cv2.destroyAllWindows()

: 

## 이미지 전처리하기

In [ ]:
proto = './semi-project/colorization_deploy_v2.prototxt'
weights = './semi-project/colorization_release_v2.caffemodel'
# weights = './semi-project/colorization_release_v2_norebal.caffemodel'

#net 이라는 변수에 딥러닝 프레임 워크를 선언한다.
net = cv2.dnn.readNetFromCaffe(proto, weights)

# 클러스터 센터를 로드
pts_in_hull = np.load('./semi-project/pts_in_hull.npy')
pts_in_hull = pts_in_hull.transpose().reshape(2, 313, 1 ,1).astype(np.float32)
net.getLayer(net.getLayerId('class8_ab')).blobs = [pts_in_hull]

net.getLayer(net.getLayerId('conv8_313_rh')).blobs = [np.full((1, 313), 2.606, np.float32)]

# 이미지 전처리하기
img = cv2.imread('./semi-project/img/1900s NewYork.jpg', cv2.IMREAD_GRAYSCALE)
img_input = img.copy()
# h, w, c = img.shape

# convert grayscale TO RGB
img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

img_rgb = img.copy()

# normlize input
img_rgb = img_rgb.astype('float32') / 255

# convert RGB to LAB
img_lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2Lab)

# only L channel to be used
img_l = img_lab[:, :, 0]

#ab의 형태로 이미지를 처리하기
input_img = cv2.resize(img_l, (224, 224))
input_img -= 50 # subtract 50 for mean-centering

#여기까지가 컴퓨터가 이해할 수 있는 언어로 출력된다.

: 

## 이미지 후처리 하기

In [ ]:
# 우리가 알아볼 수 있도록 후처리 해주기

net.setInput(cv2.dnn.blobFromImage(input_img))
pred = net.forward()[0, :, :, :].transpose((1, 2, 0))

# resize to original image shape (원본 크기로 되돌리기)
pred_resize = cv2.resize(pred, (img.shape[1], img.shape[0]))

# concatenate with original image L
pred_lab = np.concatenate([img_l[:, :, np.newaxis], pred_resize], axis = 2)

# convert LAB to RGB
pred_rgb = cv2.cvtColor(pred_lab, cv2.COLOR_Lab2RGB)
pred_rgb = pred_rgb * 255
pred_rgb = np.clip(pred_rgb, 0, 255)
pred_rgb = pred_rgb.astype('uint8')

: 

### 이미지 출력하기

In [ ]:
cv2.imshow('img', img_input)
cv2.imshow('result', pred_rgb)

cv2.waitKey()
cv2.destroyAllWindows()

: 

## 사이즈 조정

In [ ]:
dst1 = cv2.resize(img_input, dsize=(800, 600), interpolation = cv2.INTER_AREA)
dst2 = cv2.resize(pred_rgb, dsize=(800, 600), interpolation = cv2.INTER_AREA)

cv2.imshow("dst1", dst1)
cv2.imshow("dst2", dst2)

cv2.waitKey()
cv2.destroyAllWindows()

: 

### Convert Lab to BGR

In [ ]:
# convert LAB to BGR
pred_bgr = cv2.cvtColor(pred_lab, cv2.COLOR_Lab2BGR)
pred_bgr = pred_bgr * 255
pred_bgr = np.clip(pred_bgr, 0, 255)
pred_bgr = pred_bgr.astype('uint8')

# cv2.imshow('img', img_input)
# cv2.imshow('result', pred_bgr)

dst1 = cv2.resize(img_input, dsize=(800, 600), interpolation = cv2.INTER_AREA)
dst2 = cv2.resize(pred_bgr, dsize=(800, 600), interpolation = cv2.INTER_AREA)

cv2.imshow("dst1", dst1)
cv2.imshow("dst2", dst2)

cv2.waitKey()
cv2.destroyAllWindows()

: 

### RGB 채널과 BGR 채널 비교

In [ ]:
dst3 = cv2.resize(pred_rgb, dsize=(800, 600), interpolation = cv2.INTER_AREA)

cv2.imshow("BGR", dst2)
cv2.imshow("RGB", dst3)

cv2.waitKey()
cv2.destroyAllWindows()

: 

In [ ]:
cv2.imshow('img', img_input)
cv2.imshow('result', pred_bgr)

while True:
    key = cv2.waitKey()
    if key == 27:
        break
        
    elif key == ord('s'):
        cv2.imwrite('result.png', pred_bgr)
        
cv2.destroyAllWindows()

: 

## 복원 데이터 저장하기

In [ ]:
proto = './semi-project/colorization_deploy_v2.prototxt'
weights = './semi-project/colorization_release_v2.caffemodel'
# weights = './semi-project/colorization_release_v2_norebal.caffemodel'

#net 이라는 변수에 딥러닝 프레임 워크를 선언한다.
net = cv2.dnn.readNetFromCaffe(proto, weights)

# 클러스터 센터를 로드
pts_in_hull = np.load('./semi-project/pts_in_hull.npy')
pts_in_hull = pts_in_hull.transpose().reshape(2, 313, 1 ,1).astype(np.float32)
net.getLayer(net.getLayerId('class8_ab')).blobs = [pts_in_hull]

net.getLayer(net.getLayerId('conv8_313_rh')).blobs = [np.full((1, 313), 2.606, np.float32)]

# 이미지 전처리하기
img = cv2.imread('./semi-project/img/Person_Kimgoo.jpg', cv2.IMREAD_GRAYSCALE)
img_input = img.copy()

# convert BGR TO RGB
img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

img_rgb = img.copy()

# normlize input
img_rgb = img_rgb.astype('float32') / 255

# convert RGB to LAB
img_lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2Lab)

# only L channel to be used
img_l = img_lab[:, :, 0]

#ab의 형태로 이미지를 처리하기
input_img = cv2.resize(img_l, (224, 224))
input_img -= 50 # subtract 50 for mean-centering

#여기까지가 컴퓨터가 이해할 수 있는 언어로 출력된다.

# 우리가 알아볼 수 있도록 후처리 해주기

net.setInput(cv2.dnn.blobFromImage(input_img))
pred = net.forward()[0, :, :, :].transpose((1, 2, 0))

# resize to original image shape (원본 크기로 되돌리기)
pred_resize = cv2.resize(pred, (img.shape[1], img.shape[0]))
# pred_resize = cv2.resize(pred, (w, h))


# concatenate with original image L
# pred_lab = np.concatenate([img_l, pred_resize], axis = 2)
pred_lab = np.concatenate([img_l[:, :, np.newaxis], pred_resize], axis = 2)

# convert LAB to BGR
pred_bgr = cv2.cvtColor(pred_lab, cv2.COLOR_Lab2BGR)
pred_bgr = pred_bgr * 255
pred_bgr = np.clip(pred_bgr, 0, 255)
pred_bgr = pred_bgr.astype('uint8')

cv2.imshow('img', img_input)
cv2.imshow('result', pred_bgr)

while True:
    key = cv2.waitKey()
    if key == 27:
        break
        
    elif key == ord('s'):
        cv2.imwrite('image111.png', pred_bgr)
        
cv2.destroyAllWindows()

# dst1 = cv2.resize(img_input, dsize=(800, 600), interpolation = cv2.INTER_AREA)
# dst2 = cv2.resize(pred_bgr, dsize=(800, 600), interpolation = cv2.INTER_AREA)

# cv2.imshow("dst1", dst1)
# cv2.imshow("dst2", dst2)

# cv2.waitKey()
# cv2.destroyAllWindows()

: 

# 전체 코드

In [ ]:
import cv2
import numpy as np

proto = './semi-project/colorization_deploy_v2.prototxt'
weights = './semi-project/colorization_release_v2.caffemodel'
# weights = './semi-project/colorization_release_v2_norebal.caffemodel'

#net 이라는 변수에 딥러닝 프레임 워크를 선언한다.
net = cv2.dnn.readNetFromCaffe(proto, weights)

# 클러스터 센터를 로드
pts_in_hull = np.load('./semi-project/pts_in_hull.npy')
pts_in_hull = pts_in_hull.transpose().reshape(2, 313, 1 ,1).astype(np.float32)
net.getLayer(net.getLayerId('class8_ab')).blobs = [pts_in_hull]

net.getLayer(net.getLayerId('conv8_313_rh')).blobs = [np.full((1, 313), 2.606, np.float32)]

# 이미지 전처리하기
img = cv2.imread('./semi-project/img/Person_Audrey Hepburn.jpg', cv2.IMREAD_GRAYSCALE)
img_input = img.copy()
# h, w, c = img.shape

# convert BGR TO RGB
img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

img_rgb = img.copy()

# normlize input
img_rgb = img_rgb.astype('float32') / 255

# convert RGB to LAB
img_lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2Lab)

# only L channel to be used
img_l = img_lab[:, :, 0]

#ab의 형태로 이미지를 처리하기
input_img = cv2.resize(img_l, (224, 224))
input_img -= 50 # subtract 50 for mean-centering

#여기까지가 컴퓨터가 이해할 수 있는 언어로 출력된다.

# 우리가 알아볼 수 있도록 후처리 해주기

net.setInput(cv2.dnn.blobFromImage(input_img))
pred = net.forward()[0, :, :, :].transpose((1, 2, 0))

# resize to original image shape (원본 크기로 되돌리기)
pred_resize = cv2.resize(pred, (img.shape[1], img.shape[0]))
# pred_resize = cv2.resize(pred, (w, h))


# concatenate with original image L
# pred_lab = np.concatenate([img_l, pred_resize], axis = 2)
pred_lab = np.concatenate([img_l[:, :, np.newaxis], pred_resize], axis = 2)

# convert LAB to BGR
pred_bgr = cv2.cvtColor(pred_lab, cv2.COLOR_Lab2BGR)
pred_bgr = pred_bgr * 255
pred_bgr = np.clip(pred_bgr, 0, 255)
pred_bgr = pred_bgr.astype('uint8')

dst1 = cv2.resize(img_input, dsize=(600, 800), interpolation = cv2.INTER_AREA)
dst2 = cv2.resize(pred_bgr, dsize=(600, 800), interpolation = cv2.INTER_AREA)

cv2.imshow("dst1", dst1)
cv2.imshow("dst2", dst2)

cv2.waitKey()
cv2.destroyAllWindows()

: 

### 고민 
1. 해상도를 복원하여 컬러화 하면 더 잘 나오지 않을까 (EDSR)
2. GAN을 이용해서 B&W 사진을 딥러닝하면 얼마나 개선된 이미지가 나올까 (GAN)

# 오래된 이미지 해상도 향상하고 색 복원하기
    - Anaconda prompt(관리자 설정에서) pip install opencv-contrib-python
    - EDSR 이라는 모델을 사용하기 위함 (이미지 해상도를 3~4배 가까이 향상)

만약에 안되면

    - pip unistall opencv-python  
    - pip uinstall opencv-contrib-python
    - pip install opencv-contrib-python
    - pip install opencv-contrib-python upgrade
    
### 결론적으로
#### result_img = sr . upsample(img) 부분에서 해결되지 않는것 같음

## EDSR(Enhanced deep super-resoution network)
    -Residual Neural Network(RL) : 잔차 신경망, 즉 레즈넷은 스킵 연결을 통해 잔차를 학습하도록 만들어진 인공신경망 

In [ ]:
import cv2
cv2.__version__

: 

In [ ]:
# 패키지 로드하기
import cv2
import numpy as np

# 모델 로드하기
proto = './semi-project/colorization_deploy_v2.prototxt'
weights = './semi-project/colorization_release_v2.caffemodel'

# net 이라는 변수에 딥러닝 프레임 워크를 선언한다.
net = cv2.dnn.readNetFromCaffe(proto, weights)

sr = cv2.dnn_superres.DnnSuperResImpl_create()
sr.readModel('./semi-project/EDSR_x4.pb')
sr.setModel('edsr', 1)

pts_in_hull = np.load('./semi-project/pts_in_hull.npy')
pts_in_hull = pts_in_hull.transpose().reshape(2, 313, 1, 1).astype(np.float32)
net.getLayer(net.getLayerId('class8_ab')).blobs = [pts_in_hull]

net.getLayer(net.getLayerId('conv8_313_rh')).blobs = [np.full((1, 313), 2.606, np.float32)]

img = cv2.imread('./semi-project/img/1900s NewYork.jpg')

#이미지 추론하기 (해당 함수는 전처리와 후처리를 한꺼번에 해준다.)
result_img = sr.upsample(img)

h, w, c = result_img.shape

img_input = result_img.copy()

img_input = img_input.astype('float32') / 255.

# 이미지 형태 변경
img_lab = cv2.cvtColor(img_input, cv2.COLOR_BGR2Lab)

#1 채널만 걸러내기, ab의 형태로 이미지를 처리하기
img_1 = img_lab[:, :, 0:1]

blob = cv2.dnn.blobFromImage(img_1, size = (224, 224), mean = [50, 50, 50])

net.setInput(blob)
output = net.forward()
# 여기까지가 컴퓨터가 이해할 수 있는 언어로 출력된다.

# 우리가 알아볼 수 있도록 후처리 해주기
output = output.squeeze().transpose((1, 2, 0))

output_resized = cv2.resize(output, (w, h))

# 1과 ab를 합쳐서 1ab 시스템의 이미지로 만들어주기
output_lab = np.concatenate([img_1, output_resized], axis = 2)

output_bgr = cv2.cvtColor(output_lab, cv2.COLOR_Lab2BGR)
output_bgr = output_bgr * 255
output_bgr = np.clip(output_bgr, 0, 255)
output_bgr = output_bgr.astype('uint8')

# 이미지 출력하기
dst1 = cv2.resize(img, dsize=(800, 600), interpolation = cv2.INTER_AREA)
dst2 = cv2.resize(output_bgr, dsize=(800, 600), interpolation = cv2.INTER_AREA)

cv2.imshow("dst1", dst1)
cv2.imshow("dst2", dst2)

cv2.waitKey()
cv2.destroyAllWindows()

: 